In [1]:
import pandas as pd
import numpy as np
import requests

# from helper import EducationData_API, ccd_data_class

In [2]:
discipline_df = pd.read_csv('data/crdc_discipline/race_2015_discipline.csv', dtype={'leaid': str})
discipline_df = discipline_df[pd.notnull(discipline_df['leaid'])]
discipline_df['leaid'] = discipline_df['leaid'].apply(lambda x: str(x).split('.')[0] if '.' in x else x)
discipline_df['leaid'] = discipline_df['leaid'].apply(lambda x: '0{}'.format(x) if len(x) == 6 else x)

keep_col = [
    'leaid',
    'students_susp_in_sch',
    'students_susp_out_sch_single',
    'students_susp_out_sch_multiple',
    'expulsions_no_ed_serv',
    'expulsions_with_ed_serv',
    'expulsions_zero_tolerance',
    'students_corporal_punish',
    'students_arrested',
    'students_referred_law_enforce'
]

for col in keep_col:
    if col != 'leaid':
        discipline_df.loc[discipline_df[col] < 0, col] = 0
discipline_df = discipline_df.groupby('leaid').agg(sum)

discipline_df.reset_index(inplace=True)
discipline_df = discipline_df[keep_col]

enrollment_df = pd.read_csv('data/crdc_enrollment/race_2015_enrollment.csv', dtype={'leaid': str})
enrollment_df['leaid'] = enrollment_df['leaid'].astype(str).apply(lambda x: str(x).split('.')[0] if '.' in x else x)
enrollment_df['leaid'] = enrollment_df['leaid'].apply(lambda x: '0{}'.format(x) if len(x) == 6 else x)
enrollment_df = enrollment_df[['leaid', 'enrollment_crdc']]
enrollment_df = enrollment_df.groupby('leaid').agg(sum)

discipline_df['enrollment'] = discipline_df['leaid'].apply(lambda x: enrollment_df.loc[x, 'enrollment_crdc'])

directory_df = pd.read_csv('data/ccd_directory/2015_directory.csv', dtype=str)
directory_df = directory_df[
    [
        'leaid', 
        'county_code', 
        'fips'
    ]
]

directory_df['county_code'] = directory_df['county_code'].apply(
    lambda x: 
    '0{}'.format(x) if len(str(x)) == 4 else str(x)
)
directory_df['fips'] = directory_df['fips'].apply(
    lambda x: 
    '0{}'.format(str(x).split('.')[0]) 
    if len(str(x).split('.')[0]) == 1 else str(x).split('.')[0]
)
directory_df['state'] = directory_df['county_code'].apply(
    lambda x: 
    x[:2]
)

discipline_df = discipline_df.merge(directory_df, how='left', left_on='leaid', right_on='leaid')
discipline_df = discipline_df[pd.notnull(discipline_df['county_code'])]

discipline_df = discipline_df[[
    'county_code',
    'students_susp_in_sch',
    'students_susp_out_sch_single',
    'students_susp_out_sch_multiple',
    'expulsions_no_ed_serv',
    'expulsions_with_ed_serv',
    'expulsions_zero_tolerance',
    'students_corporal_punish',
    'students_arrested',
    'students_referred_law_enforce'
]].groupby('county_code').sum()

discipline_df.reset_index(inplace=True)

/Users/albertlee/opt/anaconda3/envs/siads592/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
/Users/albertlee/opt/anaconda3/envs/siads592/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
finance_df = pd.read_csv('data/ccd_finance/2015_finance.csv')
finance_keep_col = [
    'exp_current_supp_serve_total',
    'exp_current_other',
    'exp_current_food_serv',
    'enrollment_fall_responsible',
    'enrollment_fall_school',
    'salaries_total',
    'rev_fed_total',
    'salaries_food_service',
    'benefits_supp_sch_admin',
    'exp_current_sch_admin',
    'rev_fed_child_nutrition_act',
    'salaries_supp_bco',
    'rev_total',
    'exp_current_elsec_total',
    'salaries_teachers_regular_prog',
    'rev_state_total',
    'exp_total',
    'salaries_supp_sch_admin',
    'exp_current_bco',
    'rev_state_gen_formula_assist',
    'salaries_supp_operation_plant',
    'exp_current_operation_plant',
    'salaries_instruction',
    'rev_fed_state_title_i',
    'exp_current_instruction_total',
    'rev_fed_state_idea',
    'benefits_employee_total',
    'benefits_supp_bco',
    'debt_longterm_outstand_beg_fy',
    'debt_longterm_outstand_end_fy',
    'rev_local_total',
    'benefits_supp_operation_plant',
    'rev_fed_nonspec',
    'payments_private_schools',
    'salaries_teachers_vocational',
    'payments_charter_schools',
    'rev_state_bilingual_ed',
    'rev_state_gifted_talented',
    'exp_utilities_energy',
    'rev_state_vocational_ed',
    'rev_local_dist_activ_receipts',
    'assets_sinking_fund'
]

finance_df = finance_df[['leaid'] + finance_keep_col]
for col in finance_keep_col:
    finance_df.loc[finance_df[col] < 0, col] = 0
    
directory_df = pd.read_csv('data/ccd_directory/2015_directory.csv', dtype=str)
directory_df = directory_df[
    [
        'leaid', 
        'county_code', 
        'fips'
    ]
]

directory_df['county_code'] = directory_df['county_code'].apply(
    lambda x: 
    '0{}'.format(x) if len(str(x)) == 4 else str(x)
)
directory_df['fips'] = directory_df['fips'].apply(
    lambda x: 
    '0{}'.format(str(x).split('.')[0]) 
    if len(str(x).split('.')[0]) == 1 else str(x).split('.')[0]
)
directory_df['state'] = directory_df['county_code'].apply(
    lambda x: 
    x[:2]
)
    
finance_df = finance_df.merge(directory_df, how='left', left_on='leaid', right_on='leaid')
finance_df = finance_df[pd.notnull(finance_df['county_code'])]

finance_df = finance_df[[
    'county_code',
    'exp_current_supp_serve_total',
    'exp_current_other',
    'exp_current_food_serv',
    'enrollment_fall_responsible',
    'enrollment_fall_school',
    'salaries_total',
    'rev_fed_total',
    'salaries_food_service',
    'benefits_supp_sch_admin',
    'exp_current_sch_admin',
    'rev_fed_child_nutrition_act',
    'salaries_supp_bco',
    'rev_total',
    'exp_current_elsec_total',
    'salaries_teachers_regular_prog',
    'rev_state_total',
    'exp_total',
    'salaries_supp_sch_admin',
    'exp_current_bco',
    'rev_state_gen_formula_assist',
    'salaries_supp_operation_plant',
    'exp_current_operation_plant',
    'salaries_instruction',
    'rev_fed_state_title_i',
    'exp_current_instruction_total',
    'rev_fed_state_idea',
    'benefits_employee_total',
    'benefits_supp_bco',
    'debt_longterm_outstand_beg_fy',
    'debt_longterm_outstand_end_fy',
    'rev_local_total',
    'benefits_supp_operation_plant',
    'rev_fed_nonspec',
    'payments_private_schools',
    'salaries_teachers_vocational',
    'payments_charter_schools',
    'rev_state_bilingual_ed',
    'rev_state_gifted_talented',
    'exp_utilities_energy',
    'rev_state_vocational_ed',
    'rev_local_dist_activ_receipts',
    'assets_sinking_fund'
]].groupby('county_code').sum()

finance_df.reset_index(inplace=True)

In [4]:
full_df = discipline_df.merge(finance_df, how='inner', left_on='county_code', right_on='county_code')

In [5]:
incarceration_trends = pd.read_csv('data/incarcerationtrends/incarceration_trends.csv')
incarceration_trends = incarceration_trends[incarceration_trends['year'] == 2015]
incarceration_trends['total_prison_adm_rate'] = incarceration_trends['total_prison_adm_rate'] / incarceration_trends['total_pop']
incarceration_trends['incar_rate'] = incarceration_trends['total_prison_adm_rate'].fillna(0)
incarceration_trends['county_code'] = incarceration_trends['fips'].apply(
    lambda x: 
    '0{}'.format(x) if len(str(x)) == 4 else str(x)
)
incarceration_trends = incarceration_trends[['county_code','incar_rate']]

In [6]:
incarceration_trends

,county_code,incar_rate
45,01001,0.007526
94,01003,0.001392
143,01005,0.009575
192,01007,0.002313
241,01009,0.002718
...,...,...
153611,56037,0.006701
153660,56039,0.002402
153709,56041,0.007662
153758,56043,0.031338


In [7]:
full_df = full_df.merge(incarceration_trends, how='inner', left_on='county_code', right_on='county_code')

In [8]:
full_df.head()

,county_code,students_susp_in_sch,students_susp_out_sch_single,students_susp_out_sch_multiple,expulsions_no_ed_serv,expulsions_with_ed_serv,expulsions_zero_tolerance,students_corporal_punish,students_arrested,students_referred_law_enforce,...,payments_private_schools,salaries_teachers_vocational,payments_charter_schools,rev_state_bilingual_ed,rev_state_gifted_talented,exp_utilities_energy,rev_state_vocational_ed,rev_local_dist_activ_receipts,assets_sinking_fund,incar_rate
0,01001,6170.0,770.0,276.0,144.0,0.0,114.0,956.0,0.0,156.0,...,0.0,1072000,0.0,9000,0,0,0,1518000,3396000,0.007526
1,01003,8912.0,8472.0,6020.0,488.0,84.0,0.0,0.0,0.0,502.0,...,42000.0,4087000,0.0,67000,0,0,0,5651000,6171000,0.001392
2,01005,4790.0,1546.0,312.0,0.0,48.0,0.0,286.0,102.0,102.0,...,0.0,525000,0.0,9000,0,0,0,756000,0,0.009575
3,01007,1866.0,808.0,480.0,36.0,168.0,0.0,426.0,12.0,48.0,...,188000.0,605000,0.0,2000,0,0,0,1143000,1013000,0.002313
4,01009,4516.0,2364.0,194.0,24.0,0.0,0.0,2976.0,0.0,72.0,...,0.0,1291000,0.0,35000,0,0,0,3427000,3317000,0.002718


In [9]:
full_df.to_csv('data/merged_education_data.csv', index=False)